# Optimal binary search trees

## Example: searching a dictionary
Imagine looking up a word in an English-French dictionary: we could ensure an $O(\lg n)$ search time per word by using a **red-black tree** or any other balanced binary search tree (BTS). However, words we search can appear with different frequencies: for example, assigning a high-frequency word near the leaf of the tree will slow down the search. Moreover, some words we search may not appear in the BTS at all. How can we organise a BTS, so as to **minimise the number of nodes visited** in all search, given that we know the **frequency** of each word? Here is an analog:

|Words in dictionary|$K=\langle k_1,k_2,...,k_n\rangle$ of $n$ distinct keys in sorted order|
|:---------------------:|:---------------------------------------------------------------------:|
|Frequency of words in dictionary|$p=\langle p_1,p_2,...,p_n\rangle$|
|Words NOT in dictionary (dummy keys)|$d=\langle d_0,d_1,...,d_n\rangle$|
|Frequency of dummy keys|$q=\langle q_0,q_1,...,q_n\rangle$|

## Cost of search in a BTS
Every search is either succesfful (finding some key $k_i$) or unsuccessful (finding some dummy key $d_i$), so we have:
$$
\begin{align}
\sum^n_{i=1}p_i+\sum^n_{i=0}q_i=1
\end{align}
$$
We assume that the actual cost of a search equals the number of nodes examined, i.e., the depth of the node found by the search in $T$ plus $1$. The expected cost depends on both frequency of the key and **depth of the node** containing the key:
$$
\begin{align}
E[\text{search cost in }T]&=\sum^n_{i=1}(depth_T(k_i)+1)\bullet p_i+\sum^n_{i=0}(depth_T(d_i)+1)\bullet q_i\\
&=1+\sum^n_{i=1}depth_T(k_i)\bullet p_i+\sum^n_{i=0}depth_T(d_i)\bullet q_i\\
\end{align}
$$

## The optimal binary search tree problem
For a given set of probabilities, we wish to construct a BTS such that its expected search cost is **minimal**. We call such a tree an ***optimal binary search tree (oBTS)***. We shall see in the following session, that a brute-force approach to check all possibilities fails to yield an efficient algorithm; instead, we can solve it with dynamic programming. 

## Step 1: The optimal substructure of an oBTS
If an oBTS $T$ has a subtree $T'$ containing keys $k_i,...,k_j$, then this subtree $T'$ must be optimal.

## Step 2: A recursive solution
Leut us define $e[i,j]$ as the expected cost of searching an oBTS containing the keys $k_i,...,k_j$. Ultimately, we wish to compute $e[1,n]$.

### The easy case: when $j=i-1$
We have just the dummy key $d_{i-1}$, therefore the expected search cost is $e[i,i-1]=q_{i-1}$ for all $i$.

### When $j\geq i$:
We need to select a root $k_r$ from among $k_i,...,k_j$, so that its left subtree contains nodes $k_i,...,k_{r-1}$ and its right subtree contains nodes $k_{r+1},...,k_j$. What happens to the expected search cost of a subtree when itself becomes a subtree of a node? *The depth of each node in the subtree increases by $1$*. Therefore, the expected search cost of the all nodes in this subtree increases by the *sum of probabilities of all nodes* in this subtree, which can be expressed as:
$$
\begin{align}
w[i,j]=\sum^j_{l=i}p_l+\sum^j_{l=i-1}q_l
\end{align}
$$
Thus, if $k_r$ is the root of an optimal subtree containing keys $k_i,...,k_j$:

$$
\begin{align}
e[i,j]&=p_r\bullet 1+(e[i,r-1]+w[i,r-1])+(e[r+1,j]+w[r+1,j]))\\
&=e[i,r-1]+e[r+1,j]+w[i,r-1])+w[r+1,j])+p_r\\
&=e[i,r-1]+e[r+1,j]+w(i,j)\\
\end{align}
$$

The trick to derive a recurrence is by assuming that we know which node $k_r$ to use as the root, thus we have:

$$
\begin{align}
e[i,j]=\left\{
        \begin{array}{ll}
        q_{i-1} &\text{if}\ j=i-1\\
        \min \{ e[i,r-1]+e[r+1,j]+w(i,j)\} &\text{if}\ j\geq i\\
        \end{array}       
        \right.
\end{align}
$$

## Step 3: Computing the expected search cost of an oBTS
We can compute the cost $c[i,j]$ in a tabular, bottom-up format:
* We store the **cost** $e[i,j]$ values in a table $e[1..n+1,0..n]$
* We record the **choice of root** $k_r$ of the subtree containing $k_i,...k_j$ in a table $root[1..n+1,0..n]$
* For efficiency, we store the **probability** $w[i,j]$ in a table $w[1..n+1,0..n]$

Why the dimension of the three tables $e,w,root$ are all $(1..n+1)\times (0..n)$?

Remember that the keys $k_0,...,k_n$ and the dummy keys $d_0,...,d_n$ are distributed as:

$$
\begin{align}
r_n=\underbrace{}_{d_0}k_1\underbrace{}_{d_1}k_2...k_{n-1}\underbrace{}_{d_{n-1}}k_n\underbrace{}_{d_n}
\end{align}
$$

* $table[1,0]$ stores information of the dummy key $d_0$
* $table[n+1,n]$ stores information of the dummy key $d_n$

From this distribution, we can also observe the recurrence:
$$
\begin{align}
w[i,j]=\underset{\text{preceding term}}{w[i,j-1]}+\underset{P_{k_i}}{p_i}+\underset{P_{d_i}}{q_j}
\end{align}
$$

`optimal_bst` takes as input the probabilities $p_1,...,p_n$ and $q_0,...q_n$ and the size $n$, and it returns the table $e$ and $r$.

In [4]:
import numpy as np
p=np.array([0.15,0.1,0.05,0.1,0.2])
q=np.array([0.05,0.1,0.05,0.05,0.05,0.1])
n=len(p)

In [16]:
def optimal_bst(p,q,n):
    p=np.concatenate([[0],p]) #assign p[0]=0 for easy indexing
    e=np.zeros((n+2,n+1)) #contains one more row for easy indexing
    w=np.zeros((n+2,n+1)) #contains one more row for easy indexing
    root=np.zeros((n+2,n+1))
    for i in range(1,n+2): #initialisation
        e[i,i-1]=q[i-1] #frequency of d
        w[i,i-1]=q[i-1] #frequency of d
    for l in range(1,n+1): #l=j-i+1 (when j>=2)
        for i in range(1, n-l+2):
            j=i+l-1
            e[i,j]=np.inf
            w[i,j]=w[i,j-1]+p[j]+q[j]
            for r in range(i,j+1):
                update=e[i,r-1]+e[r+1,j]+w[i,j]
                if update<e[i,j]:
                    e[i,j]=update
                    root[i,j]=r
    return e,root
optimal_bst(p,q,n)

(array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.05, 0.45, 0.9 , 1.25, 1.75, 2.75],
        [0.  , 0.1 , 0.4 , 0.7 , 1.2 , 2.  ],
        [0.  , 0.  , 0.05, 0.25, 0.6 , 1.3 ],
        [0.  , 0.  , 0.  , 0.05, 0.3 , 0.9 ],
        [0.  , 0.  , 0.  , 0.  , 0.05, 0.5 ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.1 ]]),
 array([[0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 2., 2., 2.],
        [0., 0., 2., 2., 2., 4.],
        [0., 0., 0., 3., 4., 5.],
        [0., 0., 0., 0., 4., 5.],
        [0., 0., 0., 0., 0., 5.],
        [0., 0., 0., 0., 0., 0.]]))

## Analysis of oBTS
You may observe the similarity between the recurrence of oBTS and matrix-chain multiplication; similarly, `optimal_bts` also takes $\Theta (n^3)$ time. 